<a href="https://colab.research.google.com/github/Kyrrevel/pythonkurs_v25/blob/main/kode/Pythonkurs%20-%20Del%202%20-%20A%20short%20REST%20API%20-%20Plot%20temperature%20on%20map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install requests_cache

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.5/66.5 kB 2.4 MB/s eta 0:00:00


In [2]:
import requests
import requests_cache
import json
import folium

In [3]:
requests_cache.install_cache(cache_name='apitest_cache', backend='sqlite', expire_after=720)

In [25]:
import requests
import folium

yr_headers = {
    'User-agent': 'visualisere.no',
    'From': 'api@botnen.org'
}
yr_api_url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"

vin_headers = {
    'Ocp-Apim-Subscription-Key': '6571c1f8bc01483fad19545c7657ed39'
}
vin_api_url = "https://apis.vinmonopolet.no/stores/v0/details"

def vin_get_stores(vin_city):
    result_array = []
    response = requests.get(vin_api_url, headers=vin_headers)
    json_response = response.json()

    for store in json_response:
        address = dict(store['address'])
        if address['city'] == vin_city:
            result_array.append(address['gpsCoord'])
    return result_array

def yr_get_temperature(yr_lat, yr_lon):
    response = requests.get(yr_api_url, headers=yr_headers, params={"lat": yr_lat, "lon": yr_lon})
    json_response = response.json()
    data = json_response['properties']['timeseries']
    new_dict = dict(dict(data[0])['data'])
    return new_dict['instant']['details']['air_temperature']

def temperature_to_color(temperature):
    # Adjust the temperature range to suit the data
    if temperature <= 12:
        return 'blue'  # Cold temperatures
    elif temperature <= 13:
        return 'lightblue'
    elif temperature <= 14:
        return 'yellow'
    elif temperature <= 15:
        return 'orange'
    else:
        return 'red'  # Hot temperatures

def plot_folium(markers_list, desc_list):
    map = folium.Map(location=[60.39, 5.32], zoom_start=4, control_scale=True)
    i = 0
    for marker in markers_list:
        icon_lat, icon_lon = marker.split(';')
        # Use the temperature in the corresponding description list for color
        temperature = float(desc_list[i])  # Ensure it's a float

        # Determine the color based on temperature
        marker_color = temperature_to_color(temperature)

        folium.Marker(
            location=[icon_lat, icon_lon],
            popup=f"{desc_list[i]} \N{DEGREE CELSIUS}",
            icon=folium.Icon(color=marker_color, icon="info-sign"),
        ).add_to(map)
        i += 1
    return map

# Example usage
stores_list = vin_get_stores("Oslo")
temperature_list = []

if len(stores_list) > 0:
    for store in stores_list:
        lat, lon = store.split(';')
        temperature = yr_get_temperature(lat, lon)
        temperature_list.append(temperature)

    map = plot_folium(stores_list, temperature_list)
    map.save("temperature_map.html")


<ipython-input-25-07b65f8632ef>:60: UserWarning: color argument of Icon should be one of: {'beige', 'pink', 'blue', 'red', 'lightred', 'white', 'lightgray', 'lightgreen', 'darkblue', 'orange', 'cadetblue', 'gray', 'darkred', 'black', 'green', 'darkgreen', 'lightblue', 'darkpurple', 'purple'}.
  icon=folium.Icon(color=marker_color, icon="info-sign"),


In [26]:
map